<a href="https://colab.research.google.com/github/eododukale/93_Python_Data_Analytics_Projects/blob/main/LinearRegressionIMPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fd30812341c517c73b7ef41c1977baa6c16ab7f823e11d4c17b49d6eda7a55fa
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FINDING CREW MEMBERS FOR MY SHIPING BUSINESS  LINEAR REGRESSION").getOrCreate()

In [4]:
df = spark.read.csv("/cruise_ship_info.csv", header = True, inferSchema = True)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

# New Section

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.dtypes

[('Ship_name', 'string'),
 ('Cruise_line', 'string'),
 ('Age', 'int'),
 ('Tonnage', 'double'),
 ('passengers', 'double'),
 ('length', 'double'),
 ('cabins', 'double'),
 ('passenger_density', 'double'),
 ('crew', 'double')]

In [8]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)


In [9]:
indexed.show()
indexed.dtypes

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

[('Ship_name', 'string'),
 ('Cruise_line', 'string'),
 ('Age', 'int'),
 ('Tonnage', 'double'),
 ('passengers', 'double'),
 ('length', 'double'),
 ('cabins', 'double'),
 ('passenger_density', 'double'),
 ('crew', 'double'),
 ('cruise_cat', 'double')]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["Age", "Tonnage", "passengers", "length","cabins", "passenger_density", "cruise_cat" ],
    outputCol="features")

output = assembler.transform(indexed)

output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [11]:
final_data = output.select("features", "crew")

In [12]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [13]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean| 7.61809090909092|
| stddev|3.437945165958406|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [14]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               48|
|   mean|8.197708333333333|
| stddev|3.654012216019905|
|    min|              1.6|
|    max|             21.0|
+-------+-----------------+



In [15]:
from pyspark.ml.regression import LinearRegression

ship_lr = LinearRegression(featuresCol = 'features', labelCol='crew')

ship_model = ship_lr.fit(train_data)

In [16]:
type(ship_model)

pyspark.ml.regression.LinearRegressionModel

In [17]:
ship_results = ship_model.evaluate(test_data)

print(ship_results.r2)

0.9383733813987596


In [18]:
unlabled_data = test_data.select("features")

unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,133.5,39.59,...|
|[6.0,113.0,37.82,...|
|[7.0,116.0,31.0,9...|
|[7.0,158.0,43.7,1...|
|[8.0,77.499,19.5,...|
|[9.0,81.0,21.44,9...|
|[9.0,110.0,29.74,...|
|[9.0,113.0,26.74,...|
|[10.0,46.0,7.0,6....|
|[10.0,68.0,10.8,7...|
|[10.0,86.0,21.14,...|
|[10.0,105.0,27.2,...|
|[11.0,58.6,15.66,...|
|[11.0,90.0,22.4,9...|
|[11.0,91.62700000...|
|[12.0,25.0,3.88,5...|
|[12.0,50.0,7.0,7....|
|[12.0,77.104,20.0...|
|[12.0,91.0,20.32,...|
+--------------------+
only showing top 20 rows



In [19]:
predictions = ship_model.transform(unlabled_data)

In [20]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.292880642945025|
|[5.0,133.5,39.59,...|13.076439332942758|
|[6.0,113.0,37.82,...| 11.63420952690026|
|[7.0,116.0,31.0,9...|12.729116535471945|
|[7.0,158.0,43.7,1...|13.862926841340116|
|[8.0,77.499,19.5,...| 8.535961942317485|
|[9.0,81.0,21.44,9...| 9.467348791687387|
|[9.0,110.0,29.74,...|12.167020254953755|
|[9.0,113.0,26.74,...|11.377232703360637|
|[10.0,46.0,7.0,6....|2.6773775086008653|
|[10.0,68.0,10.8,7...| 6.444623163199466|
|[10.0,86.0,21.14,...| 9.603968627315659|
|[10.0,105.0,27.2,...|11.331479496957353|
|[11.0,58.6,15.66,...| 7.339750633292238|
|[11.0,90.0,22.4,9...| 9.985177588928275|
|[11.0,91.62700000...| 9.104257372500511|
|[12.0,25.0,3.88,5...|2.7981799960434457|
|[12.0,50.0,7.0,7....| 4.344219678973551|
|[12.0,77.104,20.0...| 8.757532656868777|
|[12.0,91.0,20.32,...| 9.117121331091663|
+--------------------+------------